<a href="https://colab.research.google.com/github/Margo-10/Hakaton/blob/main/model1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
!pip install pdfminer.six

!pip install python-docx

!sudo apt install tesseract-ocr
!pip install pytesseract

!pip install pillow

!pip install PyPDF2

!sudo apt-get update
!sudo apt-get install -y poppler-utils

!pip install pdf2image

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 32 not upgraded.
Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured f

In [39]:
import os
import re
from pdfminer.high_level import extract_text as pdfminer_extract
import pytesseract
from PIL import Image
import pandas as pd
import docx
from docx import Document

from PyPDF2 import PdfReader
import warnings
from pdf2image import convert_from_path
from PyPDF2 import PdfReader, PdfWriter

import zipfile
import os
import shutil


In [23]:
!gdown 1xbA3glwHCcU5RjDEvuqgZxWNzec2MRs9

Downloading...
From (original): https://drive.google.com/uc?id=1xbA3glwHCcU5RjDEvuqgZxWNzec2MRs9
From (redirected): https://drive.google.com/uc?id=1xbA3glwHCcU5RjDEvuqgZxWNzec2MRs9&confirm=t&uuid=e741dce7-26b8-4f18-be52-d49b833f75ad
To: /content/hackaton_main.zip
100% 786M/786M [00:15<00:00, 49.9MB/s]


In [31]:
with zipfile.ZipFile('/content/hackaton_main.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/hackaton')





In [38]:
os.makedirs("/content/tmp", exist_ok=True)  # Папка для временных файлов

In [32]:
data_dir = '/content/hackaton/hackaton'

In [40]:


def check_cropbox(file_path):
    with open(file_path, 'rb') as f:
        reader = PdfReader(f)

        for page in reader.pages:
            if '/CropBox' not in page:
                warnings.warn("CropBox missing, using MediaBox") # чекните
    return pdfminer_extract(file_path)


def fix_cropbox(file_path):
    reader = PdfReader(file_path)
    writer = PdfWriter()

    for page in reader.pages:
        if '/CropBox' not in page:
            page.cropbox = page.mediabox
        writer.add_page(page)

    fixed_path = os.path.join(os.path.dirname(file_path), "fixed_" + os.path.basename(file_path))
    with open(fixed_path, 'wb') as f:
        writer.write(f)
    return fixed_path



In [41]:

def pdf(file_path):
    try:
        fixed_path = fix_cropbox(file_path)
        text = pdfminer_extract(fixed_path)

        if not text.strip():
            print("Текст пустой, запускаем OCR...")
            return pdf_c_ocr(fixed_path)
        return text

    except Exception as e:
        print(f"Ошибка PDF: {e}")
        return ""


def pdf_c_ocr(file_path):
    try:

        images = convert_from_path(file_path, dpi=300)
        text = ""
        for i, img in enumerate(images):
            try:
                text += pytesseract.image_to_string(img, lang='rus+eng') + "\n"
            except Exception as ocr_error:
                print(f"OCR ошибка на странице {i + 1}: {ocr_error}")
        return text
    except Exception as e:
        print(f"Ошибка конвертации PDF в изображение: {e}")
        return ""


In [42]:
def doc(file_path):
    try:
        print(f"Конвертация DOC в DOCX: {file_path}")
        # Убедитесь, что libreoffice установлен
        !sudo apt-get install -y libreoffice
        # Конвертируем файл
        !libreoffice --headless --convert-to docx --outdir /content/tmp {file_path}

        new_path = os.path.splitext(file_path)[0] + ".docx"
        return docx_text(new_path)
    except Exception as e:
        print(f"Ошибка конвертации DOC: {e}")
        return ""

def docx_text(file_path):
    """Чтение DOCX файлов"""
    try:
        doc = Document(file_path)
        return "\n".join([para.text for para in doc.paragraphs])

    except Exception as e:
        print(f"Ошибка чтения DOCX: {e}")
        return ""

In [43]:

def excel(file_path):
    try:
        text = ""
        dfs = pd.read_excel(file_path, sheet_name=None)

        for sheet, df in dfs.items():
            df = df.map(lambda x: '' if pd.isna(x) else x)
            text += f"Лист: {sheet}\n{df.to_string()}\n\n"
        return text

    except Exception as e:
        print(f"Excel ошибка: {e}")
        return ""


def all_text(file_path):
    if file_path.lower().endswith('.pdf'):
        return pdf(file_path)

    elif file_path.lower().endswith('.docx'):
        return docx_text(file_path)

    elif file_path.lower().endswith('.doc'):
        return doc(file_path)

    elif file_path.lower().endswith('.xlsx'):
        return excel(file_path)

    else:
        print(f"Неизвестный формат файла: {file_path}")
        return ""


def clean_text(text):
    return re.sub(r'\s+', ' ', text).strip()



In [ ]:

def main():

    if not os.path.exists(data_dir):
        print(f"Ошибка: папки {data_dir} не существует!")
        return

    file_paths = [os.path.join(data_dir, f) for f in os.listdir(data_dir)
                  if f.lower().endswith(('.pdf', '.docx', '.doc', '.xlsx'))]

    if not file_paths:
        print("В указанной папке нет файлов поддерживаемых форматов")
        return

    print(f"\nНайдено {len(file_paths)} файлов для обработки в папке {data_dir}:")

    for i, fp in enumerate(file_paths, 1):
        print(f"{i}. {os.path.basename(fp)}")

    documents = []

    for file_path in file_paths:
        filename = os.path.basename(file_path)
        print(f"\nОбработка файла: {filename}")

        try:
            text = all_text(file_path)
            if text:
                cleaned = clean_text(text)
                preview = cleaned[:100] + ('...' if len(cleaned) > 100 else '')
                documents.append({
                    'name': filename,
                    'text': cleaned,
                    'preview': preview
                })
                print(f"Успешно извлечено. Превьюшка: {preview}")

            else:
                print("Не удалось извлечь текст")
                documents.append({
                    'name': filename,
                    'text': '',
                    'preview': 'Нет текста'
                })

        except Exception as e:
            print(f"Ошибка обработки: {e}")
            documents.append({
                'name': filename,
                'text': '',
                'preview': f'Ошибка обработки: {str(e)}'
            })

    # Вывод полных результатов
    print("\n" + "=" * 50)
    print("Полные результаты обработки:")

    for doc in documents:
        print(f"\nФайл: {doc['name']}")
        print(f"Превьюшенька (первые 100 символов): {doc['preview']}")
        print("-" * 50)

    # Поиск по ключевым словам
    while True:
        query = input("\nВведите ключевые слова для поиска (или 'q' для выхода): ").strip()
        if query.lower() == 'q':
            break

        if not query:
            continue

        keywords = re.sub(r'[^\w\s]', '', query).lower().split()
        results = [d for d in documents
                   if all(kw in d['text'].lower() for kw in keywords)]

        if results:
            print(f"\nНайдено {len(results)} совпадений:")
            for res in results:
                print(f"- {res['name']}")
                print(f"  Совпадение: {res['preview']}")
        else:
            print("Совпадений не найдено")

    print("\nПрограмма завершена.")

    shutil.rmtree("/content/tmp", ignore_errors=True)  # В конце main()


if __name__ == "__main__":
    main()


Найдено 25 файлов для обработки в папке /content/hackaton/hackaton:
1. fixed_Фролов_К_В_Динамика_и_прочность_машин_МЭ_том_I_3_книга_1_1994.pdf
2. Улисс.doc
3. Фролов_К_В_Динамика_и_прочность_машин_МЭ_том_I_3_книга_1_1994.pdf
4. MOCK_DATA-7.xlsx
5. MOCK_DATA-5.xlsx
6. Фролов_К_В_Двигатели_внутреннего_сгорания_МЭ_том_IV_14_2013.pdf
7. MOCK_DATA-9.xlsx
8. Завтрак-у-Тиффани.doc
9. Фролов_К_В_Горные_машины_МЭ_том_IV_24_2010.pdf
10. MOCK_DATA.xlsx
11. MOCK_DATA-10.xlsx
12. MOCK_DATA-6.xlsx
13. 06_Великие_музеи_мира_Эрмитаж_Часть_1_2011.pdf
14. fixed_Фролов_К_В_Двигатели_внутреннего_сгорания_МЭ_том_IV_14_2013.pdf
15. MOCK_DATA-4.xlsx
16. 04_Великие_музеи_мира_Египетский_музей_2011.pdf
17. MOCK_DATA-2.xlsx
18. MOCK_DATA-3.xlsx
19. 05_Великие_музеи_мира_Метрополитен_2011.pdf
20. MOCK_DATA-8.xlsx
21. 03_Великие_музеи_мира_Лувр_Париж_2011.pdf
22. 02_Великие_музеи_мира_Прадо_Мадрид_2011.pdf
23. Фролов_К_В_Авиационные_двигатели_МЭ_том_IV_21_книга_3_2010.pdf
24. Приваловские-миллионы.doc
25. Хладно

In [1]:

!apt-get install git -y




Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.12).
0 upgraded, 0 newly installed, 0 to remove and 32 not upgraded.
